## Import packages

In [3]:
import pandas as pd
from CFModel import CFModel
from CFModel import DeepModel

/Users/joujun/.pyenv/versions/2.7.10/envs/please_my_fridge/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define constants


In [4]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 1000

## Load Recipe data

In [7]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipeid = ratings['recipeid'].drop_duplicates().max() + 1
print(len(ratings), 'ratings loaded.')

(1048575, 'ratings loaded.')


In [8]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print(len(users), 'descriptions of', max_userid, 'users loaded.')

(75061, 'descriptions of', 12936, 'users loaded.')


In [9]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid', 'title', 'categories'])
print(len(recipes), 'descriptions of', max_recipes, 'recipes loaded.')

(5101, 'descriptions of', 5023, 'recipes loaded.')


## Make recommendations for a given user

In [10]:
trained_model = DeepModel(max_userid, max_recipeid, K_FACTORS)

CFModel.py:40: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='concat'))


In [11]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

ValueError: Dimension 0 in both shapes must be equal, but are 12936 and 4. Shapes are [12936,120] and [4,120]. for 'Assign' (op: 'Assign') with input shapes: [12936,120], [4,120].

In [12]:
users[users['userid'] == TEST_USER]

,userid,nickname
1000,1000,dislab1000


In [14]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid, recipeid)

In [15]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [16]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction,title,categories
0,1000,2949,5,-0.040305,a,a
1,1000,876,5,-0.067312,a,a
2,1000,1104,5,-0.069767,a,a
3,1000,1080,5,-0.063025,a,a
4,1000,1060,5,-0.092189,a,a
5,1000,1034,5,-0.082262,a,a
6,1000,1029,5,-0.039864,a,a
7,1000,1010,5,-0.015463,a,a
8,1000,991,5,-0.070076,a,a
9,1000,984,5,-0.043135,a,a


In [17]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction,title,categories
0,4211,0.037082,a,a
1,3101,0.035626,a,a
2,4298,0.033563,a,a
3,2876,0.027424,a,a
4,560,0.024750,a,a
5,2306,0.021699,a,a
6,4742,0.021607,a,a
7,3953,0.020927,a,a
8,1047,0.019632,a,a
9,1962,0.018933,a,a
